# Deep Q-Learning for Lunar Landing

## Part 0 - Installing the required packages and importing libraries

### Installing Gymnasium

In [ ]:
%pip install gymnasium
%pip install "gymnasium[atari, accept-rom-license]"
%pip install swig
%pip install box2d pygame

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 7.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


### Importing the libraries

In [1]:
import os
import random
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple


## Part 1 - Building the AI

### Creating the architecture of the Neural Network

In [2]:
class Network(nn.Module):

    def __init__(self, state_size, action_size, seed = 42) -> None:
        super(Network, self).__init__()
        self.seed = torch.manual_seed(seed)
        self.fc1 = nn.Linear(state_size, 64)            # first full connection between input layer and first fully connected layer
        self.fc2 = nn.Linear(64, 64)                    # second full connection                       
        self.fc3 = nn.Linear(64, action_size)           # second fully connected layer to output (what action should be taken)

    def forward(self, state):        
        x = self.fc1(state)         # propagate signal from the input layer to the first fully
        x = F.relu(x)               # connected layer with a rectifier activation function
        
        x = self.fc2(x)             # propagate signal from the first fully connected layer to the second fully connected layer
        x = F.relu(x)

        return self.fc3(x)          # return the output layer containting the possible actions


## Part 2 - Training the AI

### Setting up the environment

In [3]:
import gymnasium as gym
env = gym.make('LunarLander-v3')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
number_actions = env.action_space.n
print('State shape: ', state_shape)
print('State size: ', state_size)
print('Number of actions: ', number_actions)

State shape:  (8,)
State size:  8
Number of actions:  4


### Initializing the hyperparameters

In [4]:
learning_rate = 5e-4            # Why? Trial and error experimentation
minibatch_size = 100            # Common practice for deep q-learning
discount_factor = 0.99          # Close to 1, so that the agent considers future rewards instead of being shortsighted
replay_buffer_size = int(1e5)   # Memory of the agent (How many experiences, states, rewards etc are stored in the agent)
interpolation_param = 1e-3      # Again, experimentally determined to be optimal **Look up how it is determined**

### Implementing Experience Replay

In [5]:
class ReplayMemory(object):

    def __init__(self, capacity):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.capacity = capacity
        self.memory = []
    
    # Method to add experiences to the replay memory buffer, while checking that the memory is not exceeded
    def push(self, event):
        self.memory.append(event)
        if len(self.memory) > self.capacity:
            del self.memory[0]
            # Note to self: I think using linked lists instead of a list for memory would be a lot more efficient here... look it up

    # Method to randomly select/sample a batch of experiences from the replay memory buffer
    def sample(self, batch_size):
        experiences = random.sample(self.memory, k = batch_size)
        states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
        actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device) 
        rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
        next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
        dones = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
        return states, next_states, actions, rewards, dones
        


### Implementing the DQN class

In [6]:
class Agent():

    def __init__(self, state_size, action_size):
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.state_size = state_size
        self.action_size = action_size
        self.local_qnetwork = Network(state_size, action_size).to(self.device)
        self.target_qnetwork = Network(state_size, action_size).to(self.device)
        self.optimizer = optim.Adam(self.local_qnetwork.parameters(), lr = learning_rate)
        self.memory = ReplayMemory(replay_buffer_size)
        self.t_step = 0

    def step(self, state, action, reward, next_state, done):
        self.memory.push((state, action, reward, next_state, done))
        self.t_step = (self.t_step + 1) % 4
        if self.t_step == 0:
            if len(self.memory.memory) > minibatch_size:
                experiences = self.memory.sample(100)
                self.learn(experiences, discount_factor)

    def act(self, state, epsilon = 0.):
        state = torch.from_numpy(state).float().unsqueeze(0).to(self.device)
        self.local_qnetwork.eval()
        with torch.no_grad():
            action_values = self.local_qnetwork(state)
        self.local_qnetwork.train()
        if random.random() > epsilon:
            return np.argmax(action_values.cpu().data.numpy())
        else:
            return random.choice(np.arange(self.action_size))
        
    def learn(self, experiences, discount_factor):
        states, next_states, actions, rewards, dones = experiences
        next_q_targets = self.target_qnetwork(next_states).detach().max(1)[0].unsqueeze(1)
        q_targets = rewards + (discount_factor * next_q_targets * (1 - dones))
        q_expected = self.local_qnetwork(states).gather(1, actions)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward() #back-propagation
        self.optimizer.step() #Perform the optimization step to update the model parameters
        # Modify the target network parameters 
        self.soft_update(self.local_qnetwork, self.target_qnetwork, interpolation_param) 

    # Softly update the target parameters with the weighted average of the local parameters and target parameters
    def soft_update(self, local_model, target_model, interpolation_parameter):
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(interpolation_parameter * local_param.data + (1.0 - interpolation_parameter) * target_param.data)
    
    # AGENT CLASS IMPLEMENTED


### Initializing the DQN Agent

In [7]:
agent = Agent(state_size, number_actions)

### Training the DQN Agent

In [10]:
number_episodes = 2000
max_t_steps_per_episode = 1000
epsilon_start_value = 1.0
epsilon_end_value = 0.01
epsilon_decay_value = 0.995
epsilon = epsilon_start_value
scores_on_100_episodes = deque(maxlen = 100)

for episode in range(1, number_episodes + 1):
    state, _ = env.reset()
    score = 0
    for t in range(max_t_steps_per_episode):
        action = agent.act(state, epsilon)
        next_state, reward, done, _, _ = env.step(action)
        agent.step(state, action, reward, next_state, done)
        state = next_state
        score += reward
        if done:
            break
    scores_on_100_episodes.append(score)
    epsilon = max(epsilon_end_value, epsilon_decay_value * epsilon)
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
    if episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    if np.mean(scores_on_100_episodes) > 200.0:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode - 100, np.mean(scores_on_100_episodes)))
        torch.save(agent.local_qnetwork.state_dict(), 'checkpoint.pth')
        break

Episode 100	Average Score: -103.69
Episode 200	Average Score: -19.356
Episode 300	Average Score: 59.046
Episode 400	Average Score: 194.34
Episode 423	Average Score: 201.04
Environment solved in 323 episodes!	Average Score: 201.04


## Part 3 - Visualizing the results

In [11]:
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gymnasium.wrappers import RecordVideo



def show_video_of_model(agent, env_name):
    env = gym.make(env_name, render_mode = 'rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = agent.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps = 30, format = 'FFMPEG')

show_video_of_model(agent, 'LunarLander-v3')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (600, 400) to (608, 400) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).
